### Un ultimo uso dei decoratori: caching (o ***memoization***) di valori calcolati

In [1]:
def fibonacci(n):
    assert(type(n)==int and n>=0)
    if n<=1:
        return n
    return fibonacci(n-2)+fibonacci(n-1)

In [ ]:
fibonacci(40)

In [ ]:
#fibonacci(100) non provateci!

In [9]:
def memoize(f):
    cache = {}
    def helperfun(n):
        if not cache.get(n,False):
            cache[n] = f(n)
        return cache[n]
    return helperfun

In [10]:
@memoize
def fibonacci(n):
    assert(type(n)==int and n>=0)
    if n<=1:
        return n
    return fibonacci(n-2)+fibonacci(n-1)

In [11]:
fibonacci(100)

354224848179261915075

## Classi astratte e loro uso

In [12]:
from abc import ABC, abstractmethod

In [13]:
class classe_astratta(ABC):
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        pass

### Una classe astratta non può essere istanziata

In [14]:
A = classe_astratta(10)

TypeError: Can't instantiate abstract class classe_astratta with abstract methods un_metodo_astratto

### Una sottoclasse di una classe astratta deve implementare i metodi astratti

In [21]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        pass

In [22]:
S = sottoclasse(10)

In [23]:
S.un_metodo_astratto()

In [24]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [25]:
S = sottoclasse(10)

In [26]:
S.un_metodo_astratto()

100

### Una sottoclasse di una classe astratta deve implementare ***tutti*** i metodi astratti

In [27]:
class classe_astratta(ABC):
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        pass
    
    @abstractmethod
    def un_secondo_metodo_astratto(self):
        pass

In [28]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [29]:
S = sottoclasse(10)

TypeError: Can't instantiate abstract class sottoclasse with abstract methods un_secondo_metodo_astratto

### I metodi di una classe astratta devono essere implementati da "qualche" sottoclasse, non necessariamente dalla stessa

In [30]:
class classe_astratta(ABC):
    def __init__(self, value):
        self.cached_value = value
    
    @abstractmethod
    def un_metodo_astratto(self):
        pass
    
    @abstractmethod
    def un_secondo_metodo_astratto(self):
        pass

In [31]:
class sottoclasse(classe_astratta):
    def un_metodo_astratto(self):
        return self.cached_value**2

In [32]:
class sotto_sottoclasse(sottoclasse):
    def un_secondo_metodo_astratto(self):
        pass

In [33]:
SS = sotto_sottoclasse(20)

In [34]:
SS.un_secondo_metodo_astratto()

## A che cosa serve tutto ciò?

Un caso tipico si ha quando le sottoclassi (che estendono la classe astratta) hanno più attributi e metodi comuni.

In [80]:
from abc import ABC, abstractmethod
import pygame
import math

class oggettoGrafico(ABC):
    
    black = (0,0,0)
    white = (255,255,255)
    red = (255,0,0)
    green = (0,255,0)
    blue = (0,0,255)
    
    def __init__(self, width, height, color):
        self.width = width
        self.height = height
        self.color = color
        self.display = pygame.display.set_mode((width,height))
        
    def clean(self):
        self.display.fill(oggettoGrafico.white)
    
    @staticmethod
    def ruotaO(x, y, theta):
        xp = x*math.cos(theta)-y*math.sin(theta)
        yp = x*math.sin(theta)+y*math.cos(theta)
        return xp,yp
    
    @staticmethod
    def quit():
        pygame.quit()
        
    @abstractmethod
    def sposta(self, dx, dy):
        pass
    
    @abstractmethod
    def ruota(self, theta):
        pass

In [81]:
class line(oggettoGrafico):
    
    def __init__(self, p, q, width=800, \
                 height=600, color=oggettoGrafico.black):
        super().__init__(width, height, color)
        self.x0 = p[0]
        self.y0 = p[1]
        self.x1 = q[0]
        self.y1 = q[1]
        self.disegna()
    
    def disegna(self):
        self.clean()
        pygame.draw.line(self.display, self.color, \
                         (self.x0,self.height-self.y0), (self.x1,self.height-self.y1), 3)
        pygame.display.update()
    
    def sposta(self, dx, dy, disegna=True):
        self.x0 += dx
        self.y0 += dy
        self.x1 += dx
        self.y1 += dy
        if disegna:
            self.disegna()
        
    def ruota(self, theta):
        cx = (self.x0+self.x1)/2
        cy = (self.y0+self.y1)/2
        self.sposta(-cx,-cy, disegna=False)
        self.x0,self.y0 = oggettoGrafico.ruotaO(self.x0,self.y0, theta)
        self.x1,self.y1 = oggettoGrafico.ruotaO(self.x1,self.y1, theta)
        self.sposta(cx,cy)

In [57]:
l = line((0,0),(100,200))

In [38]:
l.sposta(200,150)

In [51]:
l.ruota(math.pi/2)

In [82]:
class cerchio(oggettoGrafico):
    
    def __init__(self, c, r, width=800, \
                 height=600, color=oggettoGrafico.black):
        super().__init__(width, height, color)
        self.x0 = c[0]
        self.y0 = c[1]
        self.r = r
        self.disegna()
    
    def disegna(self):
        self.clean()
        pygame.draw.circle(self.display, self.color, \
                           (self.x0,self.height-self.y0), self.r)
        pygame.display.update()
    
    def sposta(self, dx, dy):
        self.x0 += dx
        self.y0 += dy
        self.disegna()
        
    def ruota(self, theta):
        pass

In [53]:
c = cerchio((200,200), 50)

In [54]:
c.sposta(100,300)

In [55]:
c.ruota(1)

In [273]:
oggettoGrafico.quit()

In [83]:
class rettangolo(oggettoGrafico):
    
    def __init__(self, p, w, h, width=800, \
                 height=600, color=oggettoGrafico.black):
        super().__init__(width, height, color)
        self.x0 = p[0]
        self.y0 = p[1]
        self.w = w
        self.h = h
        self.disegna()
    
    def disegna(self):
        self.clean()
        pygame.draw.rect(self.display, self.color, \
                         (self.x0,self.height-self.y0-self.h, \
                          self.w,self.h))
        pygame.display.update()
    
    def sposta(self, dx, dy, disegna=True):
        self.x0 += dx
        self.y0 += dy
        if disegna:
            self.disegna()
        
    def ruota(self, theta):
        assert theta in {math.pi/2, -math.pi/2}
        cx = self.x0+self.w/2
        cy = self.y0+self.h/2
        self.sposta(-cx,-cy, disegna=False)
        self.x0,self.y0 = oggettoGrafico.ruotaO(self.x0,self.y0, theta)
        if theta > 0:
            self.x0 -= self.h
        else:
            self.y0 -= self.w
        self.w, self.h = self.h, self.w
        self.sposta(cx,cy)

In [113]:
r = rettangolo((1,1), 150, 70, color=(124,110,87))

In [116]:
r.sposta(100,100)

In [135]:
r.ruota(math.pi/2)